In [1]:
import re
import pathlib
import pandas as pd

In [8]:
def extract_train_msg(test_msg):
    msg_split = re.split('save_every_n', test_msg)[1:]
    df = pd.DataFrame(columns=['seed', 'pretrained_ckpt', 'experiment', 'n_param', 'n_epoch', 'ade', 'fde'])
    for msg in msg_split: 
        metric = re.search("Round 0: \nTest ADE: ([\d\.]+) \nTest FDE: ([\d\.]+)", msg)
        n_epoch = re.search("Early stop at epoch ([\d]+)", msg)
        df = pd.concat([df, pd.DataFrame({
            'seed': re.search("'seed': ([\d+]),", msg).group(1),
            'pretrained_ckpt': re.search("'pretrained_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'experiment': re.search("Experiment (.*?) has started", msg).group(1),
            'n_param': re.search("The number of trainable parameters: ([\d]+)", msg).group(1),
            'n_epoch': n_epoch.group(1) if n_epoch is not None else 99,
            'ade': metric.group(1), 
            'fde': metric.group(2)}, index=[0])], ignore_index=True)
    df.seed = df.seed.astype(int)
    df.n_param = df.n_param.astype(int)
    df.n_epoch = df.n_epoch.astype(int)
    df.ade = df.ade.astype(float)
    df.fde = df.fde.astype(float)
    df['train_net'] = df['experiment'].apply(lambda x: get_train_net(x))
    df['n_train'] = df['experiment'].apply(lambda x: get_n_train(x)).astype(int)
    df['adapter_position'] = df['experiment'].apply(lambda x: get_adapter_position(x))
    # reorder columns 
    reordered_cols = ['seed', 'train_net', 'n_train', 'adapter_position', 'n_param', 'n_epoch', 'ade', 'fde', 'experiment', 'pretrained_ckpt']
    df = df.reindex(columns=reordered_cols)
    return df

def extract_test_msg(test_msg):
    msg_split = re.split('save_every_n', test_msg)[1:]
    df = pd.DataFrame(columns=['seed', 'pretrained_ckpt', 'tuned_ckpt', 'ade', 'fde'])
    for msg in msg_split: 
        metric = re.search("Round 0: \nTest ADE: ([\d\.]+) \nTest FDE: ([\d\.]+)", msg)
        df = pd.concat([df, pd.DataFrame({
            'seed': re.search("'seed': ([\d+]),", msg).group(1),
            'pretrained_ckpt': re.search("'pretrained_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'tuned_ckpt': re.search("'tuned_ckpt': '(.*?)',", msg).group(1).split('/')[1],
            'ade': metric.group(1), 
            'fde': metric.group(2)}, index=[0])], ignore_index=True)
    df.seed = df.seed.astype(int)
    df.ade = df.ade.astype(float)
    df.fde = df.fde.astype(float)
    df['train_net'] = df['tuned_ckpt'].apply(lambda x: get_train_net(x))
    df['n_train'] = df['tuned_ckpt'].apply(lambda x: get_n_train(x)).astype(int)
    df['adapter_position'] = df['tuned_ckpt'].apply(lambda x: get_adapter_position(x))
    # reorder columns 
    reordered_cols = ['seed', 'train_net', 'n_train', 'adapter_position', 'ade', 'fde', 'tuned_ckpt', 'pretrained_ckpt']
    df = df.reindex(columns=reordered_cols)
    return df

def get_train_net(ckpt_path):
    return ckpt_path.split('__')[5]

def get_n_train(ckpt_path):
    if 'adapter' in ckpt_path:
        n_train = ckpt_path.split('__')[7].split('_')[1].split('.')[0]
    elif 'weight' in ckpt_path:
        n_train = ckpt_path.split('__')[6].split('_')[1]
    else:
        n_train = ckpt_path.split('__')[6].split('_')[1].split('.')[0]
    return n_train

def get_adapter_position(ckpt_path):
    if 'adapter' in ckpt_path:
        return ckpt_path.split('__')[6]
    else:
        return None 

In [19]:
def extract_file(file_path, out_dir='../csv/log'):
    with open(file_path, 'r') as f:
        msgs = f.read()
    if 'eval' in file_path:
        df = extract_test_msg(msgs)
    else: # train
        df = extract_train_msg(msgs)
    pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
    file_name = re.search('/([^/]+).out', file_path).group(1)
    out_name = f'{out_dir}/{file_name}.csv'
    print(out_name)
    df.to_csv(out_name, index=False)


In [155]:
extract_file('../DC_TrN_80_encoder_i_eval.out')

../csv/log/DC_TrN_80_encoder_i_eval.csv


In [20]:
extract_file('../logs/DC_TrN_80_encoder_i_train.out')

DC_TrN_80_encoder_i_train
../csv/log/DC_TrN_80_encoder_i_train.csv


## Rename ckpts

In [1]:
import os 

In [8]:
old_name = '../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__adapter_parallel__0__TrN_10__AUG.pt'
new_name = old_name
new_name = new_name.replace('adapter_', '')
new_name = new_name.replace('.pt', '__lr_0.00005.pt')
name_split = new_name.split('__')
name_split[6] = 'Pos_' + name_split[6]
new_name = '__'.join(name_split)
print(old_name, new_name)

../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__adapter_parallel__0__TrN_10__AUG.pt ../ckpts/Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__parallel__Pos_0__TrN_10__AUG__lr_0.00005.pt


In [10]:
os.rename(old_name, new_name)

In [18]:
old_name = 'Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder_0-2__TrN_80.pt'
encoder_dict={
    'all_FT': 'all', 'encoder_0': 'encoder__Pos_0', 
    'encoder_1': 'encoder__Pos_1', 'encoder_2': 'encoder__Pos_2', 
    'encoder_3': 'encoder__Pos_3', 'encoder_4': 'encoder__Pos_4',
    'encoder_0-1': 'encoder__Pos_0_1', 'encoder_0-2': 'encoder__Pos_0_1_2',
    'encoder_0-3': 'encoder__Pos_0_1_2_3', 'encoder_0-4': 'encoder__Pos_0_1_2_3_4',
    'encoder_1-2': 'encoder__Pos_1_2', 'encoder_1-3': 'encoder__Pos_1_2_3',
    'encoder_1-4': 'encoder__Pos_1_2_3_4', 'encoder_2-3': 'encoder__Pos_2_3',
    'encoder_2-4': 'encoder__Pos_2_3_4', 'encoder_3-4': 'encoder__Pos_3_4'}
new_name = old_name
encoder_i = new_name.split('__')[5]
new_name = new_name.replace(encoder_i, encoder_dict[encoder_i])
print(old_name, new_name)


Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder_0-2__TrN_80.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__encoder__Pos_0_1_2__TrN_80.pt


In [21]:
old_name='Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__AUG__lr_0.00005.pt'
left=old_name.split('__AUG__')[0]
right=old_name.split('__AUG__')[1].replace('.pt', '')
new_name=f'{left}__{right}__AUG.pt'
print(old_name, new_name)

Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__AUG__lr_0.00005.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_0.00005__AUG.pt


In [23]:
old_name='Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_5e-05.pt'
new_name=old_name.replace('5e-05', '0.00005')
print(old_name, new_name)

Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_5e-05.pt Seed_1__Tr_Biker__Val_Biker__ValRatio_0.1__filter_agent_type_deathCircle_0__serial__Pos_1_2_3_4__TrN_40__lr_0.00005.pt


In [24]:
for old_name in os.listdir('../ckpts/'):
    if '5e-05' in old_name:
        left=old_name.split('__AUG__')[0]
        right=old_name.split('__AUG__')[1].replace('.pt', '')
        new_name=f'{left}__{right}__AUG.pt'
        os.rename(f'../ckpts/{old_name}', f'../ckpts/{new_name}')

In [2]:
for old_name in os.listdir('../ckpts/'):
    if 'parallelLayer' in old_name:
        new_name=old_name.replace('parallelLayer', 'parallelBlock')
        os.rename(f'../ckpts/{old_name}', f'../ckpts/{new_name}')